In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Veri setini yükle
train_data = pd.read_csv('/data/mnist_train.csv')
test_data = pd.read_csv('/data/mnist_test.csv')

# Veriyi küçült
train_data = train_data.sample(frac=0.1, random_state=1)
test_data = test_data.sample(frac=0.1, random_state=1)

# Bağımsız değişkenleri ve hedef değişkeni ayır
X_train = train_data.drop('label', axis=1).values
y_train = to_categorical(train_data['label'].values)

X_test = test_data.drop('label', axis=1).values
y_test = to_categorical(test_data['label'].values)

# Veriyi normalize et
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# Giriş boyutlarını (28, 28, 1) olarak yeniden boyutlandır ve genişlet
X_train_reshaped = X_train.reshape(-1, 28, 28, 1)
X_test_reshaped = X_test.reshape(-1, 28, 28, 1)
X_train_resized = tf.image.resize(X_train_reshaped, (224, 224))
X_test_resized = tf.image.resize(X_test_reshaped, (224, 224))
X_train_resized_rgb = np.stack((X_train_resized[:, :, :, 0],) * 3, axis=-1)
X_test_resized_rgb = np.stack((X_test_resized[:, :, :, 0],) * 3, axis=-1)

# DenseNet modelini oluştur
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model_densenet = models.Sequential()
model_densenet.add(base_model)
model_densenet.add(layers.Flatten())
model_densenet.add(layers.Dense(256, activation='relu'))
model_densenet.add(layers.Dropout(0.5))
model_densenet.add(layers.Dense(10, activation='softmax'))
model_densenet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğit
model_densenet.fit(X_train_resized_rgb, y_train, epochs=5, batch_size=64, validation_split=0.2)

# Modeli değerlendir
y_pred_densenet = model_densenet.predict(X_test_resized_rgb)
y_pred_densenet_classes = np.argmax(y_pred_densenet, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

accuracy_densenet = accuracy_score(y_test_classes, y_pred_densenet_classes)
precision_densenet = precision_score(y_test_classes, y_pred_densenet_classes, average='weighted')
recall_densenet = recall_score(y_test_classes, y_pred_densenet_classes, average='weighted')
f1_densenet = f1_score(y_test_classes, y_pred_densenet_classes, average='weighted')

print(f"Accuracy: {accuracy_densenet}")
print(f"Precision: {precision_densenet}")
print(f"Recall: {recall_densenet}")
print(f"F1 Score: {f1_densenet}")


2023-12-23 12:32:27.082475: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1204224000 exceeds 10% of free system memory.
2023-12-23 12:32:28.811977: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 200704000 exceeds 10% of free system memory.
2023-12-23 12:32:59.774305: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2890137600 exceeds 10% of free system memory.


Epoch 1/5


2023-12-23 12:33:34.680655: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2023-12-23 12:33:34.798711: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


75/75 [==============================] - 1226s 16s/step - loss: 1.7060 - accuracy: 0.8135 - val_loss: 0.4456 - val_accuracy: 0.9225
Epoch 2/5
75/75 [==============================] - 1103s 15s/step - loss: 0.4614 - accuracy: 0.8992 - val_loss: 0.9896 - val_accuracy: 0.8075
Epoch 3/5
75/75 [==============================] - 1099s 15s/step - loss: 0.3207 - accuracy: 0.9140 - val_loss: 1.3939 - val_accuracy: 0.8708
Epoch 4/5
75/75 [==============================] - 1225s 16s/step - loss: 0.2410 - accuracy: 0.9379 - val_loss: 0.6854 - val_accuracy: 0.8833
Epoch 5/5
32/32 [==============================] - 43s 1s/step
Accuracy: 0.911
Precision: 0.9332158515162203
Recall: 0.911
F1 Score: 0.9149221977768821
